In [1]:
from BinanceBackTest import *
import asyncio
import nest_asyncio
import os
import utils
from pprint import pprint
import Analysis
import DataProcess
from collections import defaultdict
from datetime import datetime

nest_asyncio.apply()

print(f"작업시작 >> {datetime.now()}")

# symbols 지정
symbols = [
    "BTCUSDT",
    "XRPUSDT",
    "ADAUSDT",
    # "NOTUSDT",
    # "SANDUSDT",
    # "ARKMUSDT",
    # "SOLUSDT",
    "DOGEUSDT",
]
# interval 지정.
# intervals = ["1m", "5m", "1h"]
intervals = ["1m", "3m", "5m", "1h"]

# 적용할 데이터의 기간 지정.
start_date = "2024-8-1"
end_date = "2024-8-7"


print(f"instance 로딩 완료 >> {datetime.now()}")
obj_process = DataProcess.TradeStopper()
obj_analy = Analysis.AnalysisManager(back_test=True)
obj_analy.intervals = intervals
obj_order = OrderManager()
obj_con = DataProcess.OrderConstraint()
obj_data = DataManager(symbols=symbols,
                       intervals=intervals,
                       start_date=start_date,
                       end_date=end_date)

# path = os.path.join(os.path.dirname(os.getcwd()), 'DataStore/kline_data.json')
# data = utils._load_json(file_path=path)
kline_data = asyncio.run(obj_data.generate_kline_interval_data(save=True))

kline_data = utils._convert_to_array(kline_data=kline_data)
kline_data = obj_data.generate_kline_closing_sync(kline_data=kline_data, save=True)
symbol_map, interval_map, data_c = utils._convert_to_container(kline_data)
obj_data.get_indices_data(data_container=data_c, lookback_days=1)

obj_analysis = Analysis.AnalysisManager()



trade_data = []

#손익비용 저장할 변수
pnl_data = defaultdict(float)
pnl_count = defaultdict(int)

#초기 투자 자본
seed_money = 1000

# 가장 지갑 생성
obj_wallet = WalletManager(initial_fund=seed_money)

작업시작 >> 2024-12-09 07:28:00.232926
instance 로딩 완료 >> 2024-12-09 07:28:00.233303
START


In [2]:
status, order_signal_open = asyncio.run(
    obj_order.generate_order_open_signal(
        symbol='XRPUSDT',
        position=1,
        # position=reset_position,
        leverage=5,
        balance=10,
        entry_price=0.5,
        open_timestamp=0,
    )
)

In [3]:
order_signal_open
obj_process.initialize_trading_data(
    symbol='XRPUSDT',
    position=order_signal_open["position"],
    entry_price=order_signal_open["entryPrice"],
)
obj_wallet.add_funds(order_signal=order_signal_open)#, fee_cost)

({'available_funds': 998.08,
  'locked_funds': {'number_of_stocks': 1,
   'profit_and_loss': 0.0,
   'maintenance_margin': 1.92,
   'profit_margin_ratio': 0},
  'total_assets': 999.9875},
 1.92)

In [4]:
pprint(obj_wallet.balance_info)
trade_data.append(order_signal_open)
trade_data.append(obj_wallet.balance_info)

{'available_funds': 998.08,
 'locked_funds': {'maintenance_margin': 1.92,
                  'number_of_stocks': 1,
                  'profit_and_loss': 0.0,
                  'profit_margin_ratio': 0},
 'total_assets': 999.9875}


In [5]:
order_signal_open

{'symbol': 'XRPUSDT',
 'tradeTimestamp': 0,
 'tradeDate': '1970-01-01 09:00:00',
 'entryPrice': 0.5,
 'currentPrice': 0.5,
 'position': 'LONG',
 'quantity': 19.2,
 'margin': 1.92,
 'breakEventPrice': 0.50025,
 'leverage': 5,
 'liq.Price': 0.45,
 'fee_cost': 0.0125,
 'memo': None}

In [10]:
obj_wallet.get_wallet_status(symbol='XRPUSDT', current_price=0.5)

{'available_funds': 998.08,
 'locked_funds': {'number_of_stocks': 1,
  'profit_and_loss': 0.0,
  'maintenance_margin': 1.92,
  'profit_margin_ratio': 0},
 'total_assets': 1000.0}

In [11]:
obj_wallet.balance_info

{'available_funds': 998.08,
 'locked_funds': {'number_of_stocks': 1,
  'profit_and_loss': 0.0,
  'maintenance_margin': 1.92,
  'profit_margin_ratio': 0},
 'total_assets': 1000.0}

In [ ]:
c.get_data('interval_3m')

In [ ]:
symbol = 'BTCUSDT'
target_interval = '3m'

symbols = list(kline_data.keys())
interval = list(kline_data[symbols[0]].keys())

result = {}
empty_data = []
for i in kline_data[symbol][target_interval]:
    open_timestamp = i[0]
    close_timestamp = i[6]
    condition = np.where((
        kline_data[symbol]['1m'][:,0]>=open_timestamp)&
    (kline_data[symbol]['1m'][:,6]<=close_timestamp))[0]
    for n in condition:
        if n == condition[-1]:
            empty_data.append(i)
            continue
        if n == condition[0]:
            new_data = [
                open_timestamp,
                kline_data[symbol]['1m'][n][1],
                kline_data[symbol]['1m'][n][2],
                kline_data[symbol]['1m'][n][3],
                kline_data[symbol]['1m'][n][4],
                kline_data[symbol]['1m'][n][5],
                close_timestamp,
                kline_data[symbol]['1m'][n][7],
                kline_data[symbol]['1m'][n][8],
                kline_data[symbol]['1m'][n][9],
                kline_data[symbol]['1m'][n][10],
                0,
            ]
        else:
            new_data = [
                open_timestamp,
                kline_data[symbol]['1m'][n][1],
                max(kline_data[symbol]['1m'][n][2], empty_data[-1][2]),
                min(kline_data[symbol]['1m'][n][3], empty_data[-1][3]),
                kline_data[symbol]['1m'][n][4],
                kline_data[symbol]['1m'][n][5] + empty_data[-1][5],
                close_timestamp,
                kline_data[symbol]['1m'][n][7] + empty_data[-1][7],
                kline_data[symbol]['1m'][n][8] + empty_data[-1][8],
                kline_data[symbol]['1m'][n][9] + empty_data[-1][9],
                kline_data[symbol]['1m'][n][10] + empty_data[-1][10],
                0,
            ]
        empty_data.append(new_data)

In [ ]:
n

In [ ]:
import utils

symbol = 'BTCUSDT'
target_interval = '3m'

symbols = list(kline_data.keys())
interval = list(kline_data[symbols[0]].keys())

result = {}

# convert_data = self.convert_kline_data_array(kline_data=kline_data)
obj = utils.DataContainer()

main_data = []
for symbol, kline_data_symbol in kline_data.items():
    a_ = [kline_data[symbol][intervals[0]]]
    for interval, _ in kline_data_symbol.items():
        if interval == intervals[0]:
            continue
        d_ = []
        for i in dummy[symbol][target_interval]:
            open_timestamp = i[0]
            close_timestamp = i[6]
            condition = np.where((
                dummy[symbol]['1m'][:,0]>=open_timestamp)&
            (dummy[symbol]['1m'][:,6]<=close_timestamp))[0]
            for n in condition:
                if n == condition[-1]:
                    empty_data.append(i)
                    continue
                if n == condition[0]:
                    new_data = [
                        open_timestamp,
                        dummy[symbol]['1m'][n][1],
                        dummy[symbol]['1m'][n][2],
                        dummy[symbol]['1m'][n][3],
                        dummy[symbol]['1m'][n][4],
                        dummy[symbol]['1m'][n][5],
                        close_timestamp,
                        dummy[symbol]['1m'][n][7],
                        dummy[symbol]['1m'][n][8],
                        dummy[symbol]['1m'][n][9],
                        dummy[symbol]['1m'][n][10],
                        0,
                    ]
                else:
                    new_data = [
                        open_timestamp,
                        dummy[symbol]['1m'][n][1],
                        max(dummy[symbol]['1m'][n][2], empty_data[-1][2]),
                        min(dummy[symbol]['1m'][n][3], empty_data[-1][3]),
                        dummy[symbol]['1m'][n][4],
                        dummy[symbol]['1m'][n][5] + empty_data[-1][5],
                        close_timestamp,
                        dummy[symbol]['1m'][n][7] + empty_data[-1][7],
                        dummy[symbol]['1m'][n][8] + empty_data[-1][8],
                        dummy[symbol]['1m'][n][9] + empty_data[-1][9],
                        dummy[symbol]['1m'][n][10] + empty_data[-1][10],
                        0,
                    ]
            # obj.set_data(data_name=f'interval_{interval}', data=[new_data])
                d_.append(new_data)
        main_data.append(d_)
    # main_data = np.array(main_data)


In [ ]:
main_data

In [ ]:
symbol = 'BTCUSDT'
target_interval = '3m'

def ex(kline_data :dict):
    symbols = list(kline_data.keys())
    intervals = list(kline_data[symbols[0]].keys())

    result = {}
    
    symbol_map = {}
    interval_map = {}
    
    
    for idx_s, (symbol, kline_data_symbol) in enumerate(kline_data.items()):
        symbol_map[symbol] = idx_s
        result[symbol] = {}
        for idx_i, (interval, kline_data_interval) in enumerate(kline_data_symbol.items()):
            interval_map[interval] = idx_i
            
            if interval == intervals[0]:
                result[symbol][interval] = kline_data_interval
                continue
            
            
            empty_data = []
            for kline_data_interval in kline_data[symbol][intervals[1:]]:
                open_timestamp = kline_data_interval[0]
                close_timestamp = kline_data_interval[6]
                condition = np.where((
                    kline_data[symbol]['1m'][:,0]>=open_timestamp)&
                (kline_data[symbol]['1m'][:,6]<=close_timestamp))[0]
                for n in condition:
                    if n == condition[-1]:
                        empty_data.append(kline_data_interval)
                        continue
                    if n == condition[0]:
                        new_data = [
                            open_timestamp,
                            kline_data[symbol]['1m'][n][1],
                            kline_data[symbol]['1m'][n][2],
                            kline_data[symbol]['1m'][n][3],
                            kline_data[symbol]['1m'][n][4],
                            kline_data[symbol]['1m'][n][5],
                            close_timestamp,
                            kline_data[symbol]['1m'][n][7],
                            kline_data[symbol]['1m'][n][8],
                            kline_data[symbol]['1m'][n][9],
                            kline_data[symbol]['1m'][n][10],
                            0,
                        ]
                    else:
                        new_data = [
                            open_timestamp,
                            kline_data[symbol]['1m'][n][1],
                            max(kline_data[symbol]['1m'][n][2], empty_data[-1][2]),
                            min(kline_data[symbol]['1m'][n][3], empty_data[-1][3]),
                            dumkline_datay[symbol]['1m'][n][4],
                            kline_data[symbol]['1m'][n][5] + empty_data[-1][5],
                            close_timestamp,
                            kline_data[symbol]['1m'][n][7] + empty_data[-1][7],
                            kline_data[symbol]['1m'][n][8] + empty_data[-1][8],
                            kline_data[symbol]['1m'][n][9] + empty_data[-1][9],
                            kline_data[symbol]['1m'][n][10] + empty_data[-1][10],
                            0,
                        ]
                    empty_data.append(new_data)
            result[symbol][interval] = empty_data
            
            
        
    
    

In [ ]:
path = os.path.join(os.path.dirname(os.getcwd()), 'DataStore/kline_data.json')

kline_data = utils._load_json(path)

def convert_to_container(kline_data):
    container_data = utils.DataContainer()
    
    map_symbol = {}
    map_interval = {}
    
    symbols = list(kline_data.keys())
    intervals = list(kline_data[symbols[0]].keys())
    
    for idx_i, interval in enumerate(intervals):
        map_interval[interval] = idx_i
    
        dummy_data = []
        
        for idx_s, symbol in enumerate(symbols):
            map_symbol[symbol] = idx_s
    
            target_data = kline_data[symbol][interval]
            dummy_data.append(target_data)
    
        dummy_data = np.array(dummy_data)
        container_data.set_data(data_name = f'interval_{interval}', data=dummy_data)
    return map_symbol, map_interval, container_data